In [ ]:
import pandas as pd
import numpy as np

In [ ]:
dataset=pd.read_csv("/content/heart.csv")

In [ ]:
dataset.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0


In [ ]:
import numpy as np
import pandas as pd


from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, VotingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import StratifiedKFold, train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler


from sklearn.metrics import classification_report, recall_score, plot_confusion_matrix, plot_precision_recall_curve, roc_curve
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns #for plotting
from sklearn.ensemble import RandomForestClassifier #for the model
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz #plot tree
from sklearn.metrics import roc_curve, auc #for model evaluation
from sklearn.metrics import classification_report #for model evaluation
from sklearn.metrics import confusion_matrix #for model evaluation
from sklearn.model_selection import train_test_split #for data splitting
import eli5 #for purmutation importance
from eli5.sklearn import PermutationImportance

In [ ]:
dataset.columns = ['age', 'sex', 'chest_pain_type', 'resting_blood_pressure', 'cholesterol', 'fasting_blood_sugar', 'rest_ecg', 'max_heart_rate_achieved',
       'exercise_induced_angina', 'st_depression', 'st_slope', 'num_major_vessels', 'thalassemia', 'target']

In [ ]:
dt=dataset

In [ ]:
dt.head()

,age,sex,chest_pain_type,resting_blood_pressure,cholesterol,fasting_blood_sugar,rest_ecg,max_heart_rate_achieved,exercise_induced_angina,st_depression,st_slope,num_major_vessels,thalassemia,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0


In [ ]:
dt['sex'][dt['sex'] == 0] = 'female'
dt['sex'][dt['sex'] == 1] = 'male'

dt['chest_pain_type'][dt['chest_pain_type'] == 0] = 'typical_angina'
dt['chest_pain_type'][dt['chest_pain_type'] == 1] = 'atypical_angina'
dt['chest_pain_type'][dt['chest_pain_type'] == 2] = 'non_anginal_pain'
dt['chest_pain_type'][dt['chest_pain_type'] == 3] = 'asymptomatic'

dt['fasting_blood_sugar'][dt['fasting_blood_sugar'] == 0] = 'lower_than_120mg'
dt['fasting_blood_sugar'][dt['fasting_blood_sugar'] == 1] = 'greater_than_120mg'

dt['rest_ecg'][dt['rest_ecg'] == 0] = 'normal'
dt['rest_ecg'][dt['rest_ecg'] == 1] = 'STT_wave_abnormality'
dt['rest_ecg'][dt['rest_ecg'] == 2] = 'left_ventricular_hypertrophy'

dt['exercise_induced_angina'][dt['exercise_induced_angina'] == 0] = 'no'
dt['exercise_induced_angina'][dt['exercise_induced_angina'] == 1] = 'yes'

dt['st_slope'][dt['st_slope'] == 0] = 'upsloping'
dt['st_slope'][dt['st_slope'] == 1] = 'flat'
dt['st_slope'][dt['st_slope'] == 2] = 'downsloping'

dt['thalassemia'][dt['thalassemia'] == 1] = 'normal'
dt['thalassemia'][dt['thalassemia'] == 2] = 'fixed_defect'
dt['thalassemia'][dt['thalassemia'] == 3] = 'reversable_defect'

In [ ]:
dt['sex'] = dt['sex'].astype('object')
dt['chest_pain_type'] = dt['chest_pain_type'].astype('object')
dt['fasting_blood_sugar'] = dt['fasting_blood_sugar'].astype('object')
dt['rest_ecg'] = dt['rest_ecg'].astype('object')
dt['exercise_induced_angina'] = dt['exercise_induced_angina'].astype('object')
dt['st_slope'] = dt['st_slope'].astype('object')
# dt['st_depression'] = dt['st_depression'].astype('object')
dt['thalassemia'] = dt['thalassemia'].astype('object')

In [ ]:
dt = pd.get_dummies(dt, drop_first=True)

In [ ]:
dt.head()

,age,resting_blood_pressure,cholesterol,max_heart_rate_achieved,st_depression,num_major_vessels,target,sex_male,chest_pain_type_atypical_angina,chest_pain_type_non_anginal_pain,chest_pain_type_typical_angina,fasting_blood_sugar_lower_than_120mg,rest_ecg_left_ventricular_hypertrophy,rest_ecg_normal,exercise_induced_angina_yes,st_slope_flat,st_slope_upsloping,thalassemia_fixed_defect,thalassemia_normal,thalassemia_reversable_defect
0,52,125,212,168,1.0,2,0,1,0,0,1,1,0,0,0,0,0,0,0,1
1,53,140,203,155,3.1,0,0,1,0,0,1,0,0,1,1,0,1,0,0,1
2,70,145,174,125,2.6,0,0,1,0,0,1,1,0,0,1,0,1,0,0,1
3,61,148,203,161,0.0,1,0,1,0,0,1,1,0,0,0,0,0,0,0,1
4,62,138,294,106,1.9,3,0,0,0,0,1,0,0,0,0,1,0,1,0,0


In [ ]:
dt.shape

(1025, 20)

In [ ]:
Xx_train, Xx_test, yy_train, yy_test = train_test_split(dt.drop('target', 1), dt['target'], test_size = .2, random_state=10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(Xx_train)
X_test = sc.transform(Xx_test)

In [ ]:
from sklearn.model_selection import GridSearchCV
treemodel1=RandomForestClassifier()
parameter={
    'n_estimators':[400,500,600],
 'criterion':['gini','entropy','log_loss'],
  # 'splitter':['best','random'],
  'max_depth':[3,4,5,6],
  'max_features':['auto', 'sqrt', 'log2']
    
}

In [ ]:
cv=GridSearchCV(treemodel1,param_grid=parameter,cv=5,scoring='accuracy')
cv.fit(X_train,yy_train)
print(cv.best_params_)
y_pred=cv.predict(X_test)
from sklearn.metrics import accuracy_score, classification_report
score=accuracy_score(y_pred,yy_test)
score

{'criterion': 'gini', 'max_depth': 6, 'max_features': 'sqrt', 'n_estimators': 500}


0.9414634146341463

In [ ]:
treemodel1 =  RandomForestClassifier(criterion='gini',max_depth=6,max_features='sqrt',n_estimators=500).fit(X_train, yy_train)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
treemodel2=DecisionTreeClassifier()
parameter={
    # 'n_estimators':[100,200,300],
 'criterion':['gini','entropy','log_loss'],
  'splitter':['best','random'],
  'max_depth':[6,7,8,9],
  'max_features':['auto', 'sqrt', 'log2']
    
}
cv=GridSearchCV(treemodel2,param_grid=parameter,cv=5,scoring='accuracy')
cv.fit(X_train,yy_train)
print(cv.best_params_)
y_pred=cv.predict(X_test)
from sklearn.metrics import accuracy_score, classification_report
score=accuracy_score(y_pred,yy_test)
score

{'criterion': 'entropy', 'max_depth': 9, 'max_features': 'log2', 'splitter': 'best'}


0.9707317073170731